In [1]:
import random
import time
import torch
from torch import nn
import pickle
import glob
import argparse
import os

torch.cuda.empty_cache()

# os.environ["CURL_CA_BUNDLE"] = ""

# parser = argparse.ArgumentParser()
# parser.add_argument('-sp','--split', type=str, choices=['val_unseen', 'val_seen', 'tests_seen', 'tests_unseen', 'others'], required=True)
# parser.add_argument('-m','--model_saved_folder_name', type=str, required=True) # folder where the model is saved
# parser.add_argument('-o','--output_name', type=str, required=True) # save the output as a pickle file to this name
# parser.add_argument('--no_appended', action='store_true')

# args = parser.parse_args()
no_appended = False
if no_appended:
    granularity = 'noappended'
    name = 'high'
else:
    granularity = 'appended'
    name = 'high_low'

/home/cz/.local/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

def accuracy(y_pred, y_batch):
    #y_pred has shape [batch, no_classes]
    maxed = torch.max(y_pred, 1)
    y_hat = maxed.indices
    num_accurate = torch.sum((y_hat == y_batch).long())
    train_accuracy = num_accurate/ y_hat.shape[0]
    return train_accuracy.item()

def accurate_both(y_pred1, y_batch1, y_pred2, y_batch2):
    #
    maxed1 = torch.max(y_pred1, 1)
    y_hat1 = maxed1.indices
    #
    maxed2 = torch.max(y_pred2, 1)
    y_hat2 = maxed2.indices
    #
    num_both_accurate = torch.sum((y_hat1 == y_batch1).long() * (y_hat2 == y_batch2).long())
    train_accuracy = num_both_accurate/ y_hat1.shape[0]
    return train_accuracy.item()


In [3]:
#Load data
import pickle
template_by_label = pickle.load(open('data/alfred_data/alfred_dicts/correct_template_by_label_ppdl.p', 'rb'))
new_labels_dict = pickle.load(open('data/alfred_data/alfred_dicts/correct_labels_dict_ppdl.p', 'rb'))
# val_set_unseen = pickle.load(open('data/alfred_data/'+ 'val_unseen_text_with_ppdl_low_appended.p', 'rb'))
val_set_unseen = pickle.load(open('../instruction2_params_val_unseen_noappended (actual).p', 'rb'))
test_set_unseen = pickle.load(open('../instruction2_params_test_unseen_appended.p', 'rb'))
val_set_seen = pickle.load(open('data/alfred_data/'+ 'val_seen_text_with_ppdl_low_appended.p', 'rb'))
val_set_seen

{'x': ['examine a bowl by the light of a desk lamp',
  'pick up a bowl and turn a lamp on',
  'pick up a brown bowl to look at under a lit lamp',
  'move a book from a desk to a sofa',
  'move the book to the couch ',
  'move the book from the table to the sofa',
  'turn on the desk lamp while holding a yellow disc',
  'grab the yellow stand on the table turn on the lamp',
  'examine the vase by lamplight',
  'to pick up a yellow object as well as turn on a lamp',
  'carry a cd while turning on a lamp',
  'take the cd from the table turn on the desk lamp',
  'put a cd on a desk',
  'examine the keys under the light of the table lamp',
  'inspect keys near a lit lamp',
  'take the keys on the dresser and turn on the lamp on the dresser',
  'look at a pillow by the light of a lamp',
  'examine the pillow in the light of the lamp on the table ',
  'pick up a pillow and turn a lamp on',
  'put a candle on the back of a toilet',
  'move a candle from a counter to the top of a toilet',
  'mo

In [4]:
#Set device
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [5]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import BertTokenizer, BertForSequenceClassification

import torch

model = T5ForConditionalGeneration.from_pretrained('t5-base')
model = model.to(device)
if no_appended:
    model.load_state_dict(torch.load('best_models/epoch_40_noappend.pt', map_location=device))
else:
    model.load_state_dict(torch.load('best_models/epoch_46_append.pt', map_location=device))
model.eval() # set model to evaluation mode

t5_tokenizer = T5Tokenizer.from_pretrained('t5-base')
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [6]:
bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
bert_model = bert_model.to(device)
bert_model.load_state_dict(torch.load('best_models/sliced.pt', map_location=device))
bert_model.eval() # set model to evaluation mode

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [7]:
# get only the high level instructions string[0:[SEP]]

test_unseen_high_level_instructions = []
test_unseen_low_level_instructions = list(test_set_unseen.keys())

for key, value in test_set_unseen.items():
    test_unseen_high_level_instructions.append(key.split('[SEP]')[0])

if no_appended:
    test_unseen = test_unseen_high_level_instructions
else:
    test_unseen = test_unseen_low_level_instructions

In [8]:
labels2task_type = pickle.load(open('data/alfred_data/alfred_dicts/label2task_type.p', 'rb'))
labels2task_type
# flip the dictionary task_type : labels
task_type2labels = {labels2task_type[k]: k for k in labels2task_type}
task_type2labels

{'pick_and_place_simple': 2,
 'look_at_obj_in_light': 5,
 'pick_and_place_with_movable_recep': 1,
 'pick_two_obj_and_place': 3,
 'pick_clean_then_place_in_recep': 6,
 'pick_heat_then_place_in_recep': 4,
 'pick_cool_then_place_in_recep': 0}

In [9]:
obj2idx = pickle.load(open('data/alfred_data/alfred_dicts/obj2idx.p', 'rb'))
recep2idx = pickle.load(open('data/alfred_data/alfred_dicts/recep2idx.p', 'rb'))
labels2task_type = pickle.load(open('data/alfred_data/alfred_dicts/label2task_type.p', 'rb'))
objects = list(obj2idx.keys())
receptacles = list(recep2idx.keys())
task_types = list(labels2task_type.values())
# change None in the lists to 'None'
objects = ['None' if x is None else x for x in objects]
receptacles = ['None' if x is None else x for x in receptacles]
objects
# # remove 'Bottle' from objects
objects.remove('Bottle')
objects.remove('LaundryHamper')
objects.remove('Poster')
objects.remove('ShowerGlass')
objects.remove('ShowerHead')
# objects = {'AlarmClock': 0, 'Apple': 1, 'AppleSliced': 2, 'BaseballBat': 3, 'BasketBall': 4, 'Book': 5, 'Bowl': 6, 'Box': 7, 'Bread': 8, 'BreadSliced': 9, 'ButterKnife': 10, 'CD': 11, 'Candle': 12, 'CellPhone': 13, 'Cloth': 14, 'CreditCard': 15, 'Cup': 16, 'DeskLamp': 17, 'DishSponge': 18, 'Egg': 19, 'Faucet': 20, 'FloorLamp': 21, 'Fork': 22, 'Glassbottle': 23, 'HandTowel': 24, 'HousePlant': 25, 'Kettle': 26, 'KeyChain': 27, 'Knife': 28, 'Ladle': 29, 'Laptop': 30, 'LaundryHamperLid': 31, 'Lettuce': 32, 'LettuceSliced': 33, 'LightSwitch': 34, 'Mug': 35, 'Newspaper': 36,
#                     'Pan': 37, 'PaperTowel': 38, 'PaperTowelRoll': 39, 'Pen': 40, 'Pencil': 41, 'PepperShaker': 42, 'Pillow': 43, 'Plate': 44, 'Plunger': 45, 'Pot': 46, 'Potato': 47, 'PotatoSliced': 48, 'RemoteControl': 49, 'SaltShaker': 50, 'ScrubBrush': 51, 'ShowerDoor': 52, 'SoapBar': 53, 'SoapBottle': 54, 'Spatula': 55, 'Spoon': 56, 'SprayBottle': 57, 'Statue': 58, 'StoveKnob': 59, 'TeddyBear': 60, 'Television': 61, 'TennisRacket': 62, 'TissueBox': 63, 'ToiletPaper': 64, 'ToiletPaperHanger':65, 'ToiletPaperRoll': 66, 'Tomato': 67, 'TomatoSliced': 68, 'Towel': 69, 'Vase': 70, 'Watch': 71, 'WateringCan': 72, 'WineBottle': 73}
# objects = list(objects.keys())
# receptacles = {'ArmChair': 0, 'BathtubBasin': 1, 'Bed': 2, 'Cabinet': 3, 'Cart': 4, 'CoffeeMachine': 5, 'CoffeeTable': 6, 'CounterTop': 7, 'Desk': 8, 'DiningTable': 9, 'Drawer': 10,
#                                'Dresser': 11, 'Fridge': 12, 'GarbageCan': 13, 'Microwave': 14, 'Ottoman': 15, 'Safe': 16, 'Shelf': 17, 'SideTable': 18, 'SinkBasin': 19, 'Sofa': 20, 'StoveBurner': 21, 'TVStand': 22, 'Toilet': 23}
# receptacles = list(receptacles.keys())
print(receptacles)
print(objects)

['None', 'TowelHolder', 'Cup', 'CoffeeMachine', 'Sink', 'Ottoman', 'CounterTop', 'Desk', 'Sofa', 'Mug', 'CoffeeTable', 'ToiletPaperHanger', 'Pot', 'TVStand', 'Cart', 'Cabinet', 'Toilet', 'DiningTable', 'Safe', 'Bathtub', 'LaundryHamper', 'HandTowelHolder', 'Dresser', 'Pan', 'Plate', 'Box', 'BathtubBasin', 'StoveBurner', 'SideTable', 'Fridge', 'GarbageCan', 'DiningTable,TVStand', 'Toaster', 'Drawer', 'Bowl', 'Shelf', 'ArmChair', 'Sink,Sink', 'Bed', 'SinkBasin', 'TVStand,CoffeeTable', 'Microwave']
['AlarmClock', 'AluminumFoil', 'Apple', 'AppleSliced', 'ArmChair', 'BaseballBat', 'BasketBall', 'Bathtub', 'BathtubBasin', 'Bed', 'Blinds', 'Book', 'Boots', 'Bowl', 'Box', 'Bread', 'BreadSliced', 'ButterKnife', 'Cabinet', 'Candle', 'CD', 'CellPhone', 'Chair', 'Cloth', 'CoffeeMachine', 'CoffeeTable', 'CounterTop', 'CreditCard', 'Cup', 'Curtains', 'Desk', 'DeskLamp', 'Desktop', 'DiningTable', 'DishSponge', 'DogBed', 'Drawer', 'Dresser', 'Dumbbell', 'Egg', 'EggCracked', 'Faucet', 'Floor', 'FloorLa

matches the generated outputs to longest common subsequence of the two strings.

In [10]:
def longest_common_substring(s1, s2):
    m, n = len(s1), len(s2)
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    longest = 0  # Length of the longest common substring
    
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if s1[i - 1] == s2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1] + 1
                longest = max(longest, dp[i][j])
            else:
                dp[i][j] = 0
    
    return longest

def find_best_match_by_contiguous_substring(generated_target, desired_targets):
    best_match = None
    best_length = 0
    
    for target in desired_targets:
        substring_length = longest_common_substring(generated_target.lower(), target.lower())
        if substring_length > best_length:
            best_length = substring_length
            best_match = target
    
    return best_match

# Example usage
generated_target = "cool"
desired_targets = ['AlarmClock',
 'AluminumFoil',
 'Apple',
 'AppleSliced',
 'ArmChair',
 'BaseballBat',
 'BasketBall',
 'Bathtub',
 'BathtubBasin',
 'Bed',
 'Blinds',
 'Book',
 'Boots',
 'Bottle',
 'Bowl',
 'Box',
 'Bread',
 'BreadSliced',
 'ButterKnife',
 'Cabinet',
 'Candle',
 'CD',
 'CellPhone',
 'Chair',
 'Cloth',
 'WateringCan',
 'Window',
 'WineBottle',
 'Glassbottle']


best_match = find_best_match_by_contiguous_substring(generated_target, task_types)
print(f"The best match is: {best_match}")


The best match is: pick_cool_then_place_in_recep


In [11]:
test_unseen_low_preds_top1 = {}
test_unseen_low_preds_top2 = {}
test_unseen_low_preds_top3 = {}
test_unseen_low_preds_top4 = {}
test_unseen_low_preds_top5 = {}

for i in range(len(test_unseen)):
    task_prefix = 'summarize the task parameters: '
    instruction = test_unseen[i]
    # for t5 model 4 parameters (task_type, object_target, mrecep_target, parent_target)
    input_ids = t5_tokenizer(task_prefix + instruction, return_tensors='pt').input_ids.to(device)
    outputs = model.generate(input_ids, max_length=700, num_beams=5, num_return_sequences=5, early_stopping=False)

    for j, output in enumerate(outputs):
        predicted_parameters = t5_tokenizer.decode(output, skip_special_tokens=True)
        # for sliced bert model
        encoding_test_unseen = bert_tokenizer(test_unseen[i], return_tensors='pt', padding=True, truncation=True, max_length=512)
        input_ids_test_unseen = encoding_test_unseen['input_ids'].to(device)
        attention_mask_test_unseen = encoding_test_unseen['attention_mask'].to(device)

        # debugging and process the predicted parameters
        print(predicted_parameters)
        try:
            gen_task_type = predicted_parameters.split('task_type:')[1].split(',')[0]
        except:
            gen_task_type = 'pick_and_place_simple'
            
        try:
            gen_object_target = predicted_parameters.split('object_target:')[1].split(',')[0]
        except:
            gen_object_target = 'None'

        try:
            gen_mrecep_target = predicted_parameters.split('mrecep_target:')[1].split(',')[0]
        except:
            gen_mrecep_target = 'None'

        try:
            gen_parent_target = predicted_parameters.split('parent_target:')[1].split(',')[0]
        except:
            gen_parent_target = 'None'

        task_type = find_best_match_by_contiguous_substring(gen_task_type, task_types)
        object_target = find_best_match_by_contiguous_substring(gen_object_target, objects)
        mrecep_target = find_best_match_by_contiguous_substring(gen_mrecep_target, objects)
        parent_target = find_best_match_by_contiguous_substring(gen_parent_target, receptacles)


        sliced = bert_model(input_ids_test_unseen, attention_mask=attention_mask_test_unseen).logits.argmax().item()
        
        if j == 0:
            test_unseen_low_preds_top1[test_unseen_low_level_instructions[i]] = {'task_type': task_type, 'mrecep_target': mrecep_target, 'object_target': object_target, 'parent_target': parent_target, 'sliced': sliced}
        elif j == 1:
            test_unseen_low_preds_top2[test_unseen_low_level_instructions[i]] = {'task_type': task_type, 'mrecep_target': mrecep_target, 'object_target': object_target, 'parent_target': parent_target, 'sliced': sliced}
        elif j == 2:
            test_unseen_low_preds_top3[test_unseen_low_level_instructions[i]] = {'task_type': task_type, 'mrecep_target': mrecep_target, 'object_target': object_target, 'parent_target': parent_target, 'sliced': sliced}
        elif j == 3:
            test_unseen_low_preds_top4[test_unseen_low_level_instructions[i]] = {'task_type': task_type, 'mrecep_target': mrecep_target, 'object_target': object_target, 'parent_target': parent_target, 'sliced': sliced}
        elif j == 4:
            test_unseen_low_preds_top5[test_unseen_low_level_instructions[i]] = {'task_type': task_type, 'mrecep_target': mrecep_target, 'object_target': object_target, 'parent_target': parent_target, 'sliced': sliced}


test_unseen_low_preds_top1


task_type: look_at_obj_in_light, object_target: Bowl, mrecep_target: None, parent_target: None
task_type: look_at_obj_in_light, object_target: Bowl, mrecep_target: None, parent_target: None
task_type: look_at_obj_in_light. object_target: Bowl, mrecep_target: None, parent_target: None
task_type: look_at_obj_in_light, object_target: Bowl, mrecep_target: None, parent_target: None.
task_type: look_at_obj_in_light, object_target: Bowl; mrecep_target: None, parent_target: None
task_type: look_at_obj_in_light, object_target: Bowl, mrecep_target: None, parent_target: None
task_type: look_at_obj_in_light, object_target: Vase, mrecep_target: None, parent_target: None
task_type: look_at_obj_in_light. object_target: Bowl, mrecep_target: None, parent_target: None
task_type: look_at_obj_in_light, object_target: Bowl, mrecep_target: None, parent_target: None
task_type: look_at_obj_in_light; object_target: Bowl; mrecep_target: None, parent_target: None
task_type: look_at_obj_in_light, object_target: B

Token indices sequence length is longer than the specified maximum sequence length for this model (534 > 512). Running this sequence through the model will result in indexing errors


task_type: pick_two_obj_and_place, object_target: Bread, mrecep_target: None, parent_target: dining table
task_type: pick_two_obj_and_place, object_target: bread, mrecep_target: None, parent_target: Dining Table
task_type: pick_two_obj_and_place, object_target: Bread, mrecep_target: None, parent_target: Dining Table
task_type: pick_two_obj_and_place, object_target: Bread, mrecep_target: None and parent_target: Dining Table
task_type: pick_two_obj_and_place, object_target: bread, mrecep_target: None, parent_target: Dining Table
task_type: pick_two_obj_and_place, object_target: Tomato, mrecep_target: None, parent_target: Dining Table
task_type: pick_two_obj_and_place, object_target: Bread, mrecep_target: None, parent_target: dining table
task_type: pick_and_place_simple, object_target: Key Chain, mrecep_target: None, parent_target: Arm Chair
task_type: pick_two_obj_and_place, object_target: Key Chain, mrecep_target: None, parent_target: Arm Chair
task_type: pick_and_place_simple; task_ty

{'examine a grey bowl in the light of a lamp[SEP]turn right and begin walking across the room then hang a right and walk up to the end table to the left of the couch[SEP]pick up the grey bowl off of the end table[SEP]turn left and take a step forward then turn right to face the lamp[SEP]turn on the small lamp on the end table': {'task_type': 'look_at_obj_in_light',
  'mrecep_target': 'None',
  'object_target': 'Bowl',
  'parent_target': 'None',
  'sliced': 0},
 'examine a bowl by the light of a lamp[SEP]turn right walk passed the coffee table and then turn left towards the green lamp on the table[SEP]pick up the bowl on the table[SEP]turn left to face the lamp[SEP]turn on the lamp': {'task_type': 'look_at_obj_in_light',
  'mrecep_target': 'None',
  'object_target': 'Bowl',
  'parent_target': 'None',
  'sliced': 0},
 'grab the grey bowl on the corner table turn on the lamp[SEP]turn right walk to the shelf turn right walk to the corner table[SEP]grab the grey bowl on the table[SEP]turn l

In [12]:
sample_ep = 0
key = list(test_unseen_low_preds_top1.keys())[sample_ep]

print('instruction:', key)
print('test_unseen_low_preds_top1:', test_unseen_low_preds_top1[key])
print('test_unseen_low_preds_top2:', test_unseen_low_preds_top2[key])
print('test_unseen_low_preds_top3:', test_unseen_low_preds_top3[key])
print('test_unseen_low_preds_top4:', test_unseen_low_preds_top4[key])
print('test_unseen_low_preds_top5:', test_unseen_low_preds_top5[key])

instruction: examine a grey bowl in the light of a lamp[SEP]turn right and begin walking across the room then hang a right and walk up to the end table to the left of the couch[SEP]pick up the grey bowl off of the end table[SEP]turn left and take a step forward then turn right to face the lamp[SEP]turn on the small lamp on the end table
test_unseen_low_preds_top1: {'task_type': 'look_at_obj_in_light', 'mrecep_target': 'None', 'object_target': 'Bowl', 'parent_target': 'None', 'sliced': 0}
test_unseen_low_preds_top2: {'task_type': 'look_at_obj_in_light', 'mrecep_target': 'None', 'object_target': 'Bowl', 'parent_target': 'None', 'sliced': 0}
test_unseen_low_preds_top3: {'task_type': 'look_at_obj_in_light', 'mrecep_target': 'None', 'object_target': 'Bowl', 'parent_target': 'None', 'sliced': 0}
test_unseen_low_preds_top4: {'task_type': 'look_at_obj_in_light', 'mrecep_target': 'None', 'object_target': 'Bowl', 'parent_target': 'None', 'sliced': 0}
test_unseen_low_preds_top5: {'task_type': 'lo

In [13]:
test_unseen_low_preds_top1_copy = test_unseen_low_preds_top1.copy()
test_unseen_low_preds_top2_copy = test_unseen_low_preds_top2.copy()
test_unseen_low_preds_top3_copy = test_unseen_low_preds_top3.copy()
test_unseen_low_preds_top4_copy = test_unseen_low_preds_top4.copy()
test_unseen_low_preds_top5_copy = test_unseen_low_preds_top5.copy()
test_unseen_low_preds = [test_unseen_low_preds_top1_copy, test_unseen_low_preds_top2_copy, test_unseen_low_preds_top3_copy, test_unseen_low_preds_top4_copy, test_unseen_low_preds_top5_copy]
# len(val_unseen_high_preds_copy)

In [14]:
test_unseen_low_preds_top1_copy

{'examine a grey bowl in the light of a lamp[SEP]turn right and begin walking across the room then hang a right and walk up to the end table to the left of the couch[SEP]pick up the grey bowl off of the end table[SEP]turn left and take a step forward then turn right to face the lamp[SEP]turn on the small lamp on the end table': {'task_type': 'look_at_obj_in_light',
  'mrecep_target': 'None',
  'object_target': 'Bowl',
  'parent_target': 'None',
  'sliced': 0},
 'examine a bowl by the light of a lamp[SEP]turn right walk passed the coffee table and then turn left towards the green lamp on the table[SEP]pick up the bowl on the table[SEP]turn left to face the lamp[SEP]turn on the lamp': {'task_type': 'look_at_obj_in_light',
  'mrecep_target': 'None',
  'object_target': 'Bowl',
  'parent_target': 'None',
  'sliced': 0},
 'grab the grey bowl on the corner table turn on the lamp[SEP]turn right walk to the shelf turn right walk to the corner table[SEP]grab the grey bowl on the table[SEP]turn l

In [15]:
for i in test_unseen_low_preds:
    for j in i:
        if i[j]['task_type'] not in task_type2labels.keys():
            print('task_type not found:', i[j])
            # print(i[j])
            i[j]['task_type'] = 2 # set random task_type
        else:
            i[j]['task_type'] = task_type2labels[i[j]['task_type']]
        # print(i[j])
        # remove spaces from object_target
        # i[j]['object_target'] = i[j]['object_target'].replace(' ', '')
        # i[j]['mrecep_target'] = i[j]['mrecep_target'].replace(' ', '')
        # i[j]['parent_target'] = i[j]['parent_target'].replace(' ', '')
        # if 'None' change to None
        if i[j]['object_target'] == 'None':
            i[j]['object_target'] = None
        if i[j]['mrecep_target'] == 'None':
            i[j]['mrecep_target'] = None
        if i[j]['parent_target'] == 'None':
            i[j]['parent_target'] = None

In [23]:
test_unseen_low_preds[0]
test_unseen_low_preds[1]
test_unseen_low_preds[2]
test_unseen_low_preds[3]
test_unseen_low_preds[4]

{'examine a grey bowl in the light of a lamp[SEP]turn right and begin walking across the room then hang a right and walk up to the end table to the left of the couch[SEP]pick up the grey bowl off of the end table[SEP]turn left and take a step forward then turn right to face the lamp[SEP]turn on the small lamp on the end table': {'task_type': 5,
  'mrecep_target': None,
  'object_target': 'TargetCircle',
  'parent_target': None,
  'sliced': 0},
 'examine a bowl by the light of a lamp[SEP]turn right walk passed the coffee table and then turn left towards the green lamp on the table[SEP]pick up the bowl on the table[SEP]turn left to face the lamp[SEP]turn on the lamp': {'task_type': 5,
  'mrecep_target': None,
  'object_target': 'TargetCircle',
  'parent_target': None,
  'sliced': 0},
 'grab the grey bowl on the corner table turn on the lamp[SEP]turn right walk to the shelf turn right walk to the corner table[SEP]grab the grey bowl on the table[SEP]turn left and walk until corner table is

In [17]:
# save the output as a pickle file
pickle.dump(test_unseen_low_preds[0], open('test_unseen_T5_All_appended_pred1.p', 'wb'))
pickle.dump(test_unseen_low_preds[1], open('test_unseen_T5_All_appended_pred2.p', 'wb'))
pickle.dump(test_unseen_low_preds[2], open('test_unseen_T5_All_appended_pred3.p', 'wb'))
pickle.dump(test_unseen_low_preds[3], open('test_unseen_T5_All_appended_pred4.p', 'wb'))
pickle.dump(test_unseen_low_preds[4], open('test_unseen_T5_All_appended_pred5.p', 'wb'))

In [18]:
len(test_unseen_low_preds)

5